In [3]:
import os
import numpy as np
import pandas as pd
from joblib import Parallel, delayed
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# ─── Configuration ─────────────────────────────────────────────────────────
FOLDS_DIR    = "cv_folds"
PLAYER_FEATS = ["PTS", "REB", "AST"]
OPP_FEATS    = ["opp_E_OFF_RATING", "opp_E_DEF_RATING", "opp_E_NET_RATING", "opp_REB_PCT"]
TARGET_COLS  = ["PTS", "REB", "AST"]
N_JOBS       = 4

# ─── Hyperparameters to try ─────────────────────────────────────────────────
hyperparams = [
    {"n_neighbors": 3,  "weights": "uniform"},
    {"n_neighbors": 3,  "weights": "distance"},
    {"n_neighbors": 5,  "weights": "uniform"},
    {"n_neighbors": 5,  "weights": "distance"},
    {"n_neighbors": 10, "weights": "uniform"},
    {"n_neighbors": 10, "weights": "distance"},
]

# ─── Sequence builders ─────────────────────────────────────────────────────
def build_train_sequences(df):
    X_seq, X_opp, Y = [], [], []
    for pid, gp in df.groupby("PLAYER_ID"):
        gp = gp.sort_values("GAME_DATE")
        p = gp[PLAYER_FEATS].values
        o = gp[OPP_FEATS].values
        for i in range(8, len(gp)):
            X_seq.append(p[i-8:i])
            X_opp.append(o[i])
            Y.append(p[i])
    return np.array(X_seq), np.array(X_opp), np.array(Y)

def build_test_sequences(train_df, test_df):
    X_seq, X_opp, Y = [], [], []
    for row in test_df.sort_values("GAME_DATE").itertuples():
        pid = row.PLAYER_ID
        hist = train_df[train_df.PLAYER_ID == pid].sort_values("GAME_DATE")
        if len(hist) < 8:
            continue
        p = hist[PLAYER_FEATS].values
        X_seq.append(p[-8:])
        o_vec = [getattr(row, f) for f in OPP_FEATS]
        X_opp.append(o_vec)
        Y.append([row.PTS, row.REB, row.AST])
    return np.array(X_seq), np.array(X_opp), np.array(Y)

# ─── Single‐fold evaluation ─────────────────────────────────────────────────
def eval_knn_fold(fold_idx, hp):
    # load data
    tr = pd.read_csv(f"{FOLDS_DIR}/fold_{fold_idx}_train.csv", parse_dates=["GAME_DATE"])
    te = pd.read_csv(f"{FOLDS_DIR}/fold_{fold_idx}_test.csv",  parse_dates=["GAME_DATE"])

    # build sequences
    X_tr_seq, X_tr_opp, y_tr = build_train_sequences(tr)
    X_te_seq, X_te_opp, y_te = build_test_sequences(tr, te)

    # flatten sequence and combine with opponent stats
    ns, ts, nf = X_tr_seq.shape
    X_tr_flat = X_tr_seq.reshape(ns, ts * nf)
    X_te_flat = X_te_seq.reshape(X_te_seq.shape[0], ts * nf)
    X_tr = np.hstack([X_tr_flat, X_tr_opp])
    X_te = np.hstack([X_te_flat, X_te_opp])

    # scale features for KNN
    scaler = StandardScaler().fit(X_tr)
    X_tr = scaler.transform(X_tr)
    X_te = scaler.transform(X_te)

    # instantiate and train KNN
    model = KNeighborsRegressor(
        n_neighbors=hp["n_neighbors"],
        weights=hp["weights"],
        n_jobs=1
    )
    model.fit(X_tr, y_tr)

    # predict & compute metrics
    preds = model.predict(X_te)
    metrics = {}
    for idx, target in enumerate(TARGET_COLS):
        y_true = y_te[:, idx]
        y_pred = preds[:, idx]
        metrics[f"{target}_rmse"] = np.sqrt(mean_squared_error(y_true, y_pred))
        metrics[f"{target}_mae"]  = mean_absolute_error(y_true, y_pred)
        metrics[f"{target}_r2"]   = r2_score(y_true, y_pred)
    return metrics

# ─── Run evaluation ─────────────────────────────────────────────────────────
all_results = []
for hp in hyperparams:
    fold_metrics = Parallel(n_jobs=N_JOBS)(
        delayed(eval_knn_fold)(fold, hp) for fold in range(1, 11)
    )
    df = pd.DataFrame(fold_metrics)
    df["fold"]        = np.arange(1, len(df) + 1)
    df["n_neighbors"] = hp["n_neighbors"]
    df["weights"]     = hp["weights"]
    all_results.append(df)

# concatenate all runs
final_df = pd.concat(all_results, ignore_index=True)

# reorder columns
cols = ["n_neighbors", "weights", "fold"] + [
    c for c in final_df.columns if c not in ("n_neighbors", "weights", "fold")
]
final_df = final_df[cols]

# display results
print(final_df)


    n_neighbors   weights  fold  PTS_rmse   PTS_mae    PTS_r2  REB_rmse  \
0             3   uniform     1  8.872576  7.183490 -0.033861  3.100137   
1             3   uniform     2  7.567688  6.010285  0.249560  3.034873   
2             3   uniform     3  7.704659  6.107333  0.174463  3.076903   
3             3   uniform     4  7.829295  6.145793  0.231184  3.109138   
4             3   uniform     5  7.923985  6.222222  0.190372  3.124522   
5             3   uniform     6  7.921725  6.220682  0.229829  3.215702   
6             3   uniform     7  7.928902  6.201852  0.171096  3.319043   
7             3   uniform     8  8.127059  6.287582  0.199494  3.097402   
8             3   uniform     9  7.830395  6.143018  0.125254  3.109259   
9             3   uniform    10  8.362624  6.558421  0.132769  3.329826   
10            3  distance     1  8.835711  7.155249 -0.025287  3.105773   
11            3  distance     2  7.573396  6.019503  0.248427  3.034754   
12            3  distance